In [6]:
from op_analytics.datasources.platform_metrics.pg_daily_pull import PostgresDailyPull

meta = PostgresDailyPull.fetch()
meta

2025-05-20 16:24:38 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=98910
2025-05-20 16:24:38 [info     ] Initialized Google Cloud SQL connector filename=client.py lineno=32 process=98910


/Users/bellaokiddy/Documents/Github/op-analytics/src/op_analytics/coreutils/postgres/client.py:112: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(data, schema=columns)


PostgresDailyPull(jobs_df=shape: (489_717, 9)
┌────────────┬───────────┬───────────┬───────────┬───┬───────────┬─────────┬───────────┬───────────┐
│ id         ┆ pipeline_ ┆ workflow_ ┆ workflow_ ┆ … ┆ name      ┆ status  ┆ dt        ┆ stopped_a │
│ ---        ┆ id        ┆ id        ┆ name      ┆   ┆ ---       ┆ ---     ┆ ---       ┆ t         │
│ str        ┆ ---       ┆ ---       ┆ ---       ┆   ┆ str       ┆ str     ┆ date      ┆ ---       │
│            ┆ str       ┆ str       ┆ str       ┆   ┆           ┆         ┆           ┆ date      │
╞════════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═════════╪═══════════╪═══════════╡
│ 1e8e624f-d ┆ ab56eee9- ┆ 1e8e624f- ┆ main      ┆ … ┆ cannon-bu ┆ success ┆ 2025-01-1 ┆ 2025-01-1 │
│ b9b-442e-8 ┆ 4b83-4c0e ┆ db9b-442e ┆           ┆   ┆ ild-test- ┆         ┆ 0         ┆ 0         │
│ 2b8-244026 ┆ -b5f9-e76 ┆ -82b8-244 ┆           ┆   ┆ vectors   ┆         ┆           ┆           │
│ …          ┆ 0b8…      ┆ 026…      ┆       

In [ ]:
from op_analytics.datasources.platform_metrics.prometheus_daily_pull import PrometheusDailyPull

meta_prom = PrometheusDailyPull.fetch()
meta_prom

In [2]:
from op_analytics.datasources.platform_metrics import execute
import os

# Override to force BQ write.
os.environ["OPLABS_ENV"] = "prod"

result = execute.execute_pull()

2025-05-20 13:19:14 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=97122
2025-05-20 13:19:14 [info     ] Initialized Google Cloud SQL connector filename=client.py lineno=32 process=97122


/Users/bellaokiddy/Documents/Github/op-analytics/src/op_analytics/coreutils/postgres/client.py:112: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(data, schema=columns)


2025-05-20 13:19:53 [info     ] Fetched from https://prometheus-blocks-prod-us-central1.grafana.net/api/prom/api/v1/query_range: 0.39 seconds filename=request.py lineno=100 process=97122
2025-05-20 13:19:53 [info     ] Prometheus range query executed duration=0.39s filename=client.py lineno=112 process=97122 query=sum(increase(nat_tests_total[1h]))
2025-05-20 13:19:54 [info     ] DRYRUN WRITE_EMPTY: Wrote 489.4Krows 12.5MB to BQ temp_upserts.platform_metrics_platform_metrics_jobs_v1_202505201019-2b08b0b8 filename=write.py lineno=249 process=97122


    MERGE `platform_metrics.platform_metrics_jobs_v1` T
    USING `temp_upserts.platform_metrics_platform_metrics_jobs_v1_202505201019-2b08b0b8` S
    ON T.date = S.date AND T.id = S.id AND T.pipeline_id = S.pipeline_id AND T.workflow_id = S.workflow_id
    WHEN MATCHED THEN
      UPDATE SET T.workflow_name = S.workflow_name, T.number = S.number, T.name = S.name, T.status = S.status, T.stopped_at = S.stopped_at
    WHEN NOT MATCHED THEN
    

In [16]:
from op_analytics.datasources.platform_metrics import execute
from op_analytics.coreutils.partitioned.dailydatawrite import write_to_prod

with write_to_prod():
    result = execute.execute_pull_1()

2025-05-20 19:54:12 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=98910
2025-05-20 19:54:12 [info     ] Initialized Google Cloud SQL connector filename=client.py lineno=32 process=98910


/Users/bellaokiddy/Documents/Github/op-analytics/src/op_analytics/coreutils/postgres/client.py:112: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(data, schema=columns)


2025-05-20 19:55:04 [info     ] Fetched from https://prometheus-blocks-prod-us-central1.grafana.net/api/prom/api/v1/query_range: 0.40 seconds filename=request.py lineno=100 process=98910
2025-05-20 19:55:04 [info     ] Prometheus range query executed duration=0.41s filename=client.py lineno=112 process=98910 query=sum(increase(nat_tests_total[1h]))
2025-05-20 19:55:04 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=98910
2025-05-20 19:55:04 [info     ] Initialized Google Cloud SQL connector filename=client.py lineno=32 process=98910


/Users/bellaokiddy/Documents/Github/op-analytics/src/op_analytics/coreutils/postgres/client.py:112: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(data, schema=columns)


2025-05-20 19:56:34 [info     ] Fetched from https://prometheus-blocks-prod-us-central1.grafana.net/api/prom/api/v1/query_range: 0.37 seconds filename=request.py lineno=100 process=98910
2025-05-20 19:56:34 [info     ] Prometheus range query executed duration=0.38s filename=client.py lineno=112 process=98910 query=sum(increase(nat_tests_total[1h]))
2025-05-20 19:56:34 [info     ] DRYRUN WRITE_TRUNCATE: Wrote 490.2Krows 12.6MB to BQ platform_metrics.platform_metrics_jobs_v1 filename=write.py lineno=249 process=98910
2025-05-20 19:56:34 [info     ] DRYRUN WRITE_TRUNCATE: Wrote 14rows 1.7KB to BQ platform_metrics.platform_metrics_prometheus_metrics_v1 filename=write.py lineno=249 process=98910


ValueError: partition value must be a date pattern: 'dt'

In [19]:
#Backfill tables [Currently troubleshooting]
#from op_analytics.coreutils.partitioned.location import DataLocation

# # Override location for local. Othersise It will default to DataLocation.LOCAL when running from laptop.
#     def mock_location():
#         return DataLocation.GCS

import os

from op_analytics.coreutils.bigquery.write import (
    overwrite_unpartitioned_table,
)
from op_analytics.coreutils.logger import structlog

from op_analytics.datasources.platform_metrics.pg_daily_pull import PostgresDailyPull
from op_analytics.datasources.platform_metrics.prometheus_daily_pull import PrometheusDailyPull
from op_analytics.coreutils.partitioned.dailydatawrite import write_to_prod

with write_to_prod():

    log = structlog.get_logger()

    # BigQuery Dataset and Tables
    BQ_DATASET = "platform_metrics"
    ANALYTICS_TABLE_PG = "platform_metrics_jobs_v1"
    ANALYTICS_TABLE_PROM = "platform_metrics_prometheus_metrics_v1"

    # current_env = OPLabsEnvironment.PROD
    def write_pg_to_bq(data: PostgresDailyPull):
        """Write to BigQuery.

        This operation will be retired soon. Data will move to GCS + Clickhouse.
        """
        jobs_df = data.jobs_df.rename({"dt": "date"})
        # jobs_df = data.jobs_df_truncated.rename({"dt": "date"})
        
        overwrite_unpartitioned_table(
            df=jobs_df,
            dataset=BQ_DATASET,
            table_name=ANALYTICS_TABLE_PG,
        )

        return {"jobs_df": len(jobs_df)}

    def write_prom_to_bq(data: PrometheusDailyPull):
        """Write to BigQuery.

        This operation will be retired soon. Data will move to GCS + Clickhouse.
        """
        metrics_df = data.metrics_df.rename({"dt": "date"})
        # metrics_df = data.metrics_df_truncated.rename({"dt": "date"})

        overwrite_unpartitioned_table(
            df=metrics_df,
            dataset=BQ_DATASET,
            table_name=ANALYTICS_TABLE_PG,
        )

        return {"metrics_df": len(metrics_df)}

    data_pg = PostgresDailyPull.fetch()
    data_prom = PrometheusDailyPull.fetch()

    summary = {}
    summary["bigquery"] = {"pg": write_pg_to_bq(data_pg), "prom": write_prom_to_bq(data_prom)}
    summary

2025-05-20 23:48:27 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=98910
2025-05-20 23:48:27 [info     ] Initialized Google Cloud SQL connector filename=client.py lineno=32 process=98910


/Users/bellaokiddy/Documents/Github/op-analytics/src/op_analytics/coreutils/postgres/client.py:112: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(data, schema=columns)


2025-05-20 23:51:24 [info     ] Fetched from https://prometheus-blocks-prod-us-central1.grafana.net/api/prom/api/v1/query_range: 1.30 seconds filename=request.py lineno=100 process=98910
2025-05-20 23:51:24 [info     ] Prometheus range query executed duration=1.31s filename=client.py lineno=112 process=98910 query=sum(increase(nat_tests_total[1h]))
2025-05-20 23:51:24 [info     ] DRYRUN WRITE_TRUNCATE: Wrote 491.5Krows 12.6MB to BQ platform_metrics.platform_metrics_jobs_v1 filename=write.py lineno=249 process=98910
2025-05-20 23:51:24 [info     ] DRYRUN WRITE_TRUNCATE: Wrote 16rows 1.7KB to BQ platform_metrics.platform_metrics_jobs_v1 filename=write.py lineno=249 process=98910


In [13]:
import os
from unittest.mock import patch

from op_analytics.coreutils.partitioned import dailydatawrite
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datasources.platform_metrics.dataaccess import PlatformMetrics

metrics = PrometheusDailyPull.fetch()
jobs = PostgresDailyPull.fetch()


def overwrite_reviews(metrics, jobs):
    """Overwrite GCS with the backfill reviews df."""
    
    # Required to allow writing data to GCS from local.
    os.environ["ALLOW_WRITE"] = "true"

    # Override location for local. Othersise It will default to DataLocation.LOCAL when running from laptop.
    def mock_location():
        return DataLocation.GCS

    with patch.object(dailydatawrite, "determine_location", mock_location):
        PlatformMetrics.JOBS.write(
            dataframe=metrics,
            sort_by=["dt", "id", "pipeline_id", "workflow_id"],
        )

        PlatformMetrics.PROMETHEUS_METRICS.write(
            dataframe=jobs,
            sort_by=["dt", "unix_time"],
        )


overwrite_reviews(metrics, jobs)

2025-05-20 19:13:50 [info     ] Fetched from https://prometheus-blocks-prod-us-central1.grafana.net/api/prom/api/v1/query_range: 0.56 seconds filename=request.py lineno=100 process=98910
2025-05-20 19:13:50 [info     ] Prometheus range query executed duration=0.56s filename=client.py lineno=112 process=98910 query=sum(increase(nat_tests_total[1h]))
2025-05-20 19:13:50 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=98910
2025-05-20 19:13:50 [info     ] Initialized Google Cloud SQL connector filename=client.py lineno=32 process=98910


/Users/bellaokiddy/Documents/Github/op-analytics/src/op_analytics/coreutils/postgres/client.py:112: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(data, schema=columns)


AttributeError: 'PrometheusDailyPull' object has no attribute 'select'